In [117]:
import pandas as pd
import numpy as np
from functools import reduce
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from matplotlib import pyplot as plt
from tqdm import tqdm
import pathlib
import datetime
import dateutil

In [130]:
def load_data(path):
    df = pd.read_excel(path)
    title = df.iloc[3]['Unnamed: 0']
    df = df.drop([2, 3])
    data_groups = {}
    current_group = df.iloc[0].values[0]

    for index, value in zip(df.iloc[0].index, df.iloc[0].values):
        if (pd.isna(value)):
            data_groups[current_group].append(index)
        else:
            current_group = value
            data_groups[current_group] = [index]

    data_dfs = []
    index = pd.to_datetime(df['Unnamed: 0'][2:-3], dayfirst=True)

    for group, cols in tqdm(zip(list(data_groups.keys())[1:], list(data_groups.values())[1:])):
        data_df = df[cols].copy()
        data_df.columns = data_df.iloc[1].values
        data_df = data_df.drop([0, 1])[:-3]
        data_df = data_df.set_index(index)
        data_df.index = pd.to_datetime(data_df.index)
        data_df.index.names = ['timestamp']
        data_dfs.append((group, data_df))
    
    return title, data_groups, data_dfs

In [100]:
path = '../data/report1.xlsx'

In [131]:
title, data_groups,data_dfs = load_data(path)

10it [00:00, 126.03it/s]


In [190]:
i = data_dfs[0][1].index.get_indexer(['2024-06-01 22:00:00'])[0]
data_dfs[0][1][i-2:i+2]

,Итого,Прямое,Обратное
timestamp,,,
2024-06-01 19:59:59,1472,680,792
2024-06-01 20:59:59,829,415,414
2024-06-01 22:00:00,NaN,NaN,NaN
2024-06-01 22:59:59,414,163,251


In [176]:
data_dfs[0][1]['Итого'].min()

12

In [ ]:
def predict(df, title):
    predicted_df = pd.DataFrame(index=df.index)
    for col in tqdm(df.columns):
        data = df[col].to_frame().sort_index()
        data['is_nan'] = data[col].isna().astype(int)
        data['group'] = (data['is_nan'].diff() == 1).cumsum()
        data = data.reset_index()
        nan_intervals = data[data['is_nan'] == 1].groupby(
            'group').agg(start=('timestamp', 'first'), end=('timestamp', 'last'))
        data = data.set_index('timestamp')
        data = data.drop(['is_nan', 'group'], axis=1)
        nan_intervals['duration'] = nan_intervals['end'] - nan_intervals['start']
        for interval in tqdm(nan_intervals.itertuples()):
            if (len(interval) == 0):
                predicted_df[f'{col} - predict'] = pd.Series(
                    index=predicted_df.index)
                continue
            start, end, duration = interval.start, interval.end, interval.duration
            train_end = data[:start].index[-2]
            train_start = (
                train_end - pd.Timedelta(days=train_end.weekday())).normalize()
            train_start = data.iloc[data.index.get_indexer(
                [train_start], method='nearest')[0]].name
            train = data[train_start:train_end]
            seasonal_periods = 24
            if len(train) < seasonal_periods * 2:
                train = pd.concat(
                    [train, data[:train_start][len(train) - (seasonal_periods*2+1):-1]]).sort_index()
            fit = ExponentialSmoothing(
                train.values,
                trend=None,
                seasonal='add',
                seasonal_periods=seasonal_periods,
                damped_trend=False,
            ).fit()
            predict_index = data.index[data.index.get_loc(
                start):data.index.get_loc(end) + 2]
            nan_counts = len(data[start:end])
            predicted_df.loc[predict_index,
                                 f'{title} - {col}: predict'] = fit.forecast(int(nan_counts + 1))
                
    return predicted_df


resulted_df = pd.DataFrame(index=data_dfs[0][1].index)
for name, df in data_dfs:
    predicted = predict(df, name)
    resulted_df = pd.concat([resulted_df, predicted], axis=1)

  0%|          | 0/3 [00:00<?, ?it/s]/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:84: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/home/nick/transportation/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:84: R

In [90]:
for name, df in data_dfs:
    print(name)

Общая интенсивность автомобилей
Легковые (до 6 м)
Малые груз. (6-9 м)
Грузовые (9-13 м)
Груз. большие (13-22 м)
Автопоезда (22-30 м)
Автобусы
Мотоциклы
Скорость, км/ч
Загрузка, %


In [91]:
# = pd.DataFrame(index=data_dfs[0][1].index)
resulted_df['Общая интенсивность автомобилей - Итого: predict'][~resulted_df['Общая интенсивность автомобилей - Итого: predict'].isna()]
# for name, df in data_dfs:
#     resulted_df = pd.concat([resulted_df, df], axis=1)
    

timestamp
2024-06-01 22:59:59     334.060280
2024-05-25 13:00:00     522.594258
2024-05-25 14:00:01     537.126889
2024-05-25 14:59:59     529.779770
2024-06-01 22:00:00     666.414741
2024-06-15 17:00:00    1748.702236
2024-06-15 17:59:59    1849.774380
2024-07-19 13:00:00    1551.489260
2024-07-19 13:59:59    1551.874992
2024-08-21 11:00:00    1660.831737
2024-08-21 11:59:59    1614.714356
2024-12-14 17:00:00    1049.982695
2024-12-14 18:00:01    1067.246061
2024-12-14 19:00:02     978.217451
2024-12-14 19:59:59     862.387308
2024-12-27 11:00:00     643.560944
2024-12-27 12:00:01     689.066962
2024-12-27 13:00:02     690.984985
2024-12-27 14:00:03     664.068617
2024-12-27 15:00:04     663.106034
2024-12-27 16:00:05     714.252336
2024-12-27 16:59:59     620.775504
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64

In [92]:
info_row = [np.nan]
for col_name, cols in zip(list(data_groups.keys())[1:], list(data_groups.values())[1:]):
    info_row += [col_name] * len(cols)

In [93]:
len(info_row), resulted_df.shape

(29, (10254, 28))

In [94]:
resulted_df = resulted_df.reset_index()
resulted_df.loc[-1] = info_row

/tmp/ipykernel_6337/4257859202.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resulted_df.loc[-1] = info_row


In [181]:
resulted_df

,Общая интенсивность автомобилей - Итого: predict,Общая интенсивность автомобилей - Прямое: predict,Общая интенсивность автомобилей - Обратное: predict,Легковые (до 6 м) - Итого: predict,Легковые (до 6 м) - Прямое: predict,Легковые (до 6 м) - Обратное: predict,Малые груз. (6-9 м) - Итого: predict,Малые груз. (6-9 м) - Прямое: predict,Малые груз. (6-9 м) - Обратное: predict,Грузовые (9-13 м) - Итого: predict,...,Автобусы - Итого: predict,Автобусы - Прямое: predict,Автобусы - Обратное: predict,Мотоциклы - Итого: predict,Мотоциклы - Прямое: predict,Мотоциклы - Обратное: predict,"Скорость, км/ч - Прямое: predict","Скорость, км/ч - Обратное: predict","Загрузка, % - Прямое: predict","Загрузка, % - Обратное: predict"
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-01 02:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 03:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 04:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 05:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-01 06:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31 19:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-31 20:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-31 21:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
resulted_df.index = resulted_df.index + 1
resulted_df = resulted_df.sort_index()

In [97]:
save_path = str(f'{pathlib.Path(path).parent}/output/{title.replace("/", "-").replace(" ", "_")}__{datetime.datetime.now().strftime("%d-%m-%Y__%H-%M-%S")}.xslx')
save_path

'../data/output/01.12.2023_01:59:59__24-03-2025__13-59-04.xslx'

In [182]:
with pd.ExcelWriter(path, mode='a', if_sheet_exists='replace') as writer:
    resulted_df = resulted_df.reset_index().rename(columns={'timestamp': 'Дата'})
    resulted_df['Дата'] = resulted_df['Дата'].astype(str)
    resulted_df.to_excel(excel_writer=writer,
                         sheet_name='predict')

In [ ]:
resulted_df = resulted_df.rename(columns={'timestamp': 'Дата'})
resulted_df['Дата'] = resulted_df['Дата'].astype(str)

In [ ]:
resulted_df

,Дата,Общая интенсивность автомобилей - Итого: predict,Общая интенсивность автомобилей - Прямое: predict,Общая интенсивность автомобилей - Обратное: predict,Легковые (до 6 м) - Итого: predict,Легковые (до 6 м) - Прямое: predict,Легковые (до 6 м) - Обратное: predict,Малые груз. (6-9 м) - Итого: predict,Малые груз. (6-9 м) - Прямое: predict,Малые груз. (6-9 м) - Обратное: predict,...,Автобусы - Итого: predict,Автобусы - Прямое: predict,Автобусы - Обратное: predict,Мотоциклы - Итого: predict,Мотоциклы - Прямое: predict,Мотоциклы - Обратное: predict,"Скорость, км/ч - Прямое: predict","Скорость, км/ч - Обратное: predict","Загрузка, % - Прямое: predict","Загрузка, % - Обратное: predict"
0,NaT,Общая интенсивность автомобилей,Общая интенсивность автомобилей,Общая интенсивность автомобилей,Легковые (до 6 м),Легковые (до 6 м),Легковые (до 6 м),Малые груз. (6-9 м),Малые груз. (6-9 м),Малые груз. (6-9 м),...,Автобусы,Автобусы,Автобусы,Мотоциклы,Мотоциклы,Мотоциклы,"Скорость, км/ч","Скорость, км/ч","Загрузка, %","Загрузка, %"
1,2023-01-12 02:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-12 03:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-12 04:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-12 05:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10250,2025-01-31 19:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10251,2025-01-31 20:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10252,2025-01-31 21:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10253,2025-01-31 22:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
with pd.ExcelWriter('../data/test.xlsx', mode='a') as writer:
    resulted_df = resulted_df.rename(columns={'timestamp': 'Дата'})
    resulted_df['Дата'] = resulted_df['Дата'].astype(str)
    resulted_df.to_excel(excel_writer=writer,
                         sheet_name='predict',)

ValueError: Sheet 'predict' already exists and if_sheet_exists is set to 'error'.